# Introduction to Deep Learning, HW 5+6

**Name: Yifan Li**

**NetID: yl506**

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name:  Yifan Li

## Problem 2: Recurrent Neural Networks (30 points)

**Here we will work on several problems in recurrent neural networks and NLP. First, you should
just play with word embeddings to get a greater understanding of what they are and how they
work (and when normalized and unnormalized vectors are better).**

**We would like to evaluate whether the word embeddings are helping us on our sentiment
analysis task. On one of the movie review datasets, do the following:**


### (i) Train an MLP off of the average word embedding to predict sentiment (as done in class) but optimize the network settings to maximize performance.

Word embeddings, or word vectors, provide a way of mapping words from a vocabulary into a low-dimensional space, where words with similar meanings are close together. Let's play around with a set of pre-trained word vectors, to get used to their properties. There exist many sets of pretrained word embeddings; here, we use ConceptNet Numberbatch, which provides a relatively small download in an easy-to-work-with format (h5).

In [0]:
# Download word vectors
from urllib.request import urlretrieve
import os
if not os.path.isfile('mini.h5'):
    print("Downloading Conceptnet Numberbatch word embeddings...")
    conceptnet_url = 'http://conceptnet.s3.amazonaws.com/precomputed-data/2016/numberbatch/17.06/mini.h5'
    urlretrieve(conceptnet_url, 'mini.h5')

To read an `h5` file, we'll need to use the `h5py` package. Below, we use the package to open the `mini.h5` file we just downloaded. We extract from the file a list of utf-8-encoded words, as well as their $300$-dimensional vectors.

In [0]:
import numpy as np
import h5py
with h5py.File('mini.h5', 'r') as f:
    all_words = [word.decode('utf-8') for word in f['mat']['axis1'][:]]
    all_embeddings = f['mat']['block0_values'][:]

 Now,  `all_words` is a list of $V$ strings (what we call our *vocabulary*), and `all_embeddings` is a $V \times 300$ matrix. The strings are of the form `/c/language_code/word` — for example, `/c/en/cat` and `/c/es/gato`.
 
We are interested only in the English words. We use Python list comprehensions to pull out the indices of the English words, then extract just the English words (stripping the six-character `/c/en/` prefix) and their embeddings.

In [0]:
english_words = [word[6:] for word in all_words if word.startswith('/c/en/')]
english_word_indices = [i for i, word in enumerate(all_words) if word.startswith('/c/en/')]
english_embedddings = all_embeddings[english_word_indices]

The magnitude of a word vector is less important than its direction; the magnitude can be thought of as representing frequency of use, independent of the semantics of the word. 
Here, we will be interested in semantics, so we normalize our vectors, dividing each by its length. 
The result is that all of our word vectors are length 1, and as such, lie on a unit circle. 
The dot product of two vectors is proportional to the cosine of the angle between them, and provides a measure of similarity (the bigger the cosine, the smaller the angle).

In [0]:
norms = np.linalg.norm(english_embedddings, axis=1)
normalized_embeddings = english_embedddings.astype('float32') / norms.astype('float32').reshape([-1, 1])

We want to look up words easily, so we create a dictionary that maps us from a word to its index in the word embeddings matrix.

In [0]:
index = {word: i for i, word in enumerate(english_words)}

 Now we are ready to measure the similarity between pairs of words. We use numpy to take dot products.

In [0]:
def similarity_score(w1, w2):
    score = np.dot(normalized_embeddings[index[w1], :], normalized_embeddings[index[w2], :])
    return score

def print_similarity(w1,w2):
    try:
        print('{0}\t{1}\t'.format(w1,w2), \
          similarity_score('{}'.format(w1), '{}'.format(w2)))
    except:
        print('One of the words is not in the dictionary.')
    return None
  
# A word is as similar with itself as possible:
print('cat\tcat\t', similarity_score('cat', 'cat'))
# Closely related words still get high scores:
print('cat\tfeline\t', similarity_score('cat', 'feline'))
print('cat\tdog\t', similarity_score('cat', 'dog'))
# Unrelated words, not so much
print('cat\tmoo\t', similarity_score('cat', 'moo'))
print('cat\tfreeze\t', similarity_score('cat', 'freeze'))

cat	cat	 1.0000001
cat	feline	 0.8199548
cat	dog	 0.590724
cat	moo	 0.0039538303
cat	freeze	 -0.030225191


Word embeddings are fun to play around with, but their primary use is that they allow us to think of words as existing in a continuous, Euclidean space; we can then use an existing arsenal of techniques for machine learning with continuous numerical data (like logistic regression or neural networks) to process text.

Let's take a look at an especially simple version of this. We'll perform *sentiment analysis* on a set of movie reviews: in particular, we will attempt to classify a movie review as positive or negative based on its text.

We will use a Simple Word Embedding Model (SWEM, Shen et al. 2018) to do so. We will represent a review as the *mean* of the embeddings of the words in the review. Then we'll train a three-layer MLP (a neural network) to classify the review as positive or negative.

Download the `movie-simple.txt` file from the repository into this directory. Each line of that file contains
1. the numeral 0 (for negative) or the numeral 1 (for positive), followed by
2. a tab (the whitespace character), and then
3. the review itself.

In [0]:
import string
remove_punct=str.maketrans('','',string.punctuation)

# This function converts a line of our data file into
# a tuple (x, y), where x is 300-dimensional representation
# of the words in a review, and y is its label.
def convert_line_to_example(line):
    # Pull out the first character: that's our label (0 or 1)
    y = int(line[0])
    # Split the line into words using Python's split() function
    words = line[2:].translate(remove_punct).lower().split()
    # Look up the embeddings of each word, ignoring words not
    # in our pretrained vocabulary.
    embeddings = [normalized_embeddings[index[w]] for w in words
                  if w in index]
    # Take the mean of the embeddings
    x = np.mean(np.vstack(embeddings), axis=0)
    return {'x': x, 'y': y, 'w':embeddings}

# Apply the function to each line in the file.
enc = 'utf-8' # This is necessary from within the singularity shell
with open("movie-simple.txt", "r", encoding=enc) as f:
    dataset = [convert_line_to_example(l) for l in f.readlines()]

 Now that we have a dataset, let's shuffle it and do a train/test split. We use a quarter of the dataset for testing, 3/4 for training (but also ensure that we have a whole number of batches in our training set, to make the code nicer later).

In [0]:
import random
random.shuffle(dataset)

batch_size = 100
total_batches = len(dataset) // batch_size
train_batches = 3 * total_batches // 4
train, test = dataset[:train_batches*batch_size], dataset[train_batches*batch_size:]

Time to build our MLP in Tensorflow. We'll use placeholders for X and y as usual.

In [0]:
import tensorflow as tf
tf.reset_default_graph()

# Placeholders for input
X = tf.placeholder(tf.float32, [None, 300]) # 300-D vectors
y = tf.placeholder(tf.float32, [None, 1]) # Binary output (T/F)

# Three-layer MLP
h1 = tf.layers.dense(X, 100, tf.nn.relu)
h2 = tf.layers.dense(h1, 20, tf.nn.relu)
logits = tf.layers.dense(h2, 1)
probabilities = tf.sigmoid(logits)

# Loss and metrics
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(tf.sigmoid(logits)), y), tf.float32))

# Training using Adam !
train_step = tf.train.AdamOptimizer().minimize(loss)

# Initialization of variables
initialize_all = tf.global_variables_initializer()

We can now begin a session and train our model. We'll train for 250 epochs. When we're finished, we'll evaluate our accuracy on all the test data.

In [0]:
sess = tf.InteractiveSession()
sess.run(initialize_all)
for epoch in range(250):
    for batch in range(train_batches):
        data = train[batch*batch_size:(batch+1)*batch_size]
        reviews = [sample['x'] for sample in data]
        labels  = [sample['y'] for sample in data]
        labels = np.array(labels).reshape([-1, 1])
        _, l, acc = sess.run([train_step, loss, accuracy], feed_dict={X: reviews, y: labels})
    if epoch % 10 == 0:
        print("Epoch", epoch, "Loss", l, "Acc", acc)
    random.shuffle(train)

# Evaluate on test set
test_reviews = [sample['x'] for sample in test]
test_labels  = [sample['y'] for sample in test]
test_labels = np.array(test_labels).reshape([-1, 1])
acc = sess.run(accuracy, feed_dict={X: test_reviews, y: test_labels})
print("Final accuracy:", acc)
sess.close()

Epoch 0 Loss 0.681476 Acc 0.64
Epoch 10 Loss 0.19830504 Acc 0.97
Epoch 20 Loss 0.10060603 Acc 0.95
Epoch 30 Loss 0.08067612 Acc 0.97
Epoch 40 Loss 0.03943902 Acc 0.99
Epoch 50 Loss 0.044562403 Acc 1.0
Epoch 60 Loss 0.02040045 Acc 1.0
Epoch 70 Loss 0.01597068 Acc 1.0
Epoch 80 Loss 0.01323399 Acc 1.0
Epoch 90 Loss 0.0072068465 Acc 1.0
Epoch 100 Loss 0.0061071985 Acc 1.0
Epoch 110 Loss 0.013492703 Acc 0.99
Epoch 120 Loss 0.0024972514 Acc 1.0
Epoch 130 Loss 0.0014498745 Acc 1.0
Epoch 140 Loss 0.0019460714 Acc 1.0
Epoch 150 Loss 0.0007931841 Acc 1.0
Epoch 160 Loss 0.0010455147 Acc 1.0
Epoch 170 Loss 0.011070423 Acc 0.99
Epoch 180 Loss 0.0008882606 Acc 1.0
Epoch 190 Loss 0.017560422 Acc 0.99
Epoch 200 Loss 0.0006506664 Acc 1.0
Epoch 210 Loss 0.001140778 Acc 1.0
Epoch 220 Loss 0.010405387 Acc 0.99
Epoch 230 Loss 0.0005293918 Acc 1.0
Epoch 240 Loss 0.0005838051 Acc 1.0
Final accuracy: 0.96350366


This model works great for such a simple dataset, but does a little less well on something more complex. movie-pang02.txt, for instance, has 2000 longer, more complex movie reviews. It's in the same format as our simple dataset. On those longer reviews, this model achieves only 60-80% accuracy. (Increasing the number of epochs to, say, 1000, does help.)

### (ii) Train a RNN from the word embeddings to predict sentiment (as done in class) and optimize the network settings to maximize performance.

In the context of deep learning, natural language is commonly modeled with Recurrent Neural Networks (RNNs).
RNNs pass the output of a neuron back to the input of the next time step of the same neuron.
These directed cycles in the RNN architecture gives them the ability to model temporal dynamics, making them particularly suited for modeling sequences (e.g. text).


In [0]:
with open("movie-simple.txt", "r",encoding=enc) as f:
    dataset = [convert_line_to_example(l) for l in f.readlines()]
import random
random.shuffle(dataset)
batch_size = 1
total_batches = len(dataset) // batch_size
train_batches = 3 * total_batches // 4
train, test = dataset[:train_batches*batch_size], dataset[train_batches*batch_size:]

In [0]:
tf.reset_default_graph()
# sizes
n_steps = None
n_inputs = 300 # 300-D vectors
n_neurons = 50
n_outputs = 1 # binary output

# Build RNN
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_outputs])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.tanh)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
y_ = tf.layers.dense(outputs[:,-1,:], n_outputs) #logits

# Loss and metrics
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=y_)
loss = tf.reduce_mean(cross_entropy)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(tf.sigmoid(y_)), y), tf.float32))

# Training
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [0]:
num_epochs = 250 # same # as part (a), BUT TOO SLOW, ACUTALLY TOOK ~40 MINs
initialize_all = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(initialize_all)
l_ma=.74
acc_ma=.5

for epoch in range(num_epochs):
    for batch in range(train_batches):
        data = train[batch*batch_size:(batch+1)*batch_size]
        reviews = np.array([sample['w'] for sample in data]).reshape([1,-1,300])
        labels  = np.array([sample['y'] for sample in data]).reshape([1,1])
        labels = np.array(labels).reshape([-1, 1])
        _, l, acc = sess.run([train_step, loss, accuracy], feed_dict={X: reviews, y: labels})
        l_ma=.99*l_ma+(.01)*l
        acc_ma=.99*acc_ma+(.01)*acc
    if epoch % 10 == 0:
        print("Epoch", epoch, "Loss", l_ma, "Acc", acc_ma)
    random.shuffle(train)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch 0 Loss 0.3573888612832883 Acc 0.8842813596940621
Epoch 10 Loss 0.11019765479581882 Acc 0.9690619773356354
Epoch 20 Loss 0.13173155836368167 Acc 0.9600977942283906
Epoch 30 Loss 0.029505709341571203 Acc 0.991241870358205
Epoch 40 Loss 0.006868877635509963 Acc 0.9986339901496604
Epoch 50 Loss 0.00264358100856324 Acc 0.9993817448549868
Epoch 60 Loss 0.004026522684435096 Acc 0.9966224546675948
Epoch 70 Loss 0.00020721015478110267 Acc 0.9999862073682555
Epoch 80 Loss 0.010600609836024849 Acc 0.9941732041121452
Epoch 90 Loss 0.0006369693345750222 Acc 0.9998326888023266
Epoch 100 Loss 0.16830466682156225 Acc 0.9551185903276994
Epoch 110 Loss 0.03868497111872263 Acc 0.9882958010740001
Epoch 120 Loss 0.03737030617026399 Acc 0.9914410601484754
Epoch 130 Loss 0.00045571810966438557 Acc 0.9998360185799864
Epoch 140 Loss 0.000163468162970757 Acc 0.9999930481999387
Epoch 150 Loss 0.0006911297575595828 Acc 0.9995130627859232
Epoch 160 Loss 0.0006516873042593447 Acc 0.9999503993038104
Epoch 170 

In [0]:
# Evaluate on test set
test_acc=0
n=0
for sample in test:
    test_reviews = np.array([sample['w'] ]).reshape([1,-1,300])
    test_labels  = np.array([sample['y']]).reshape([1,1])
    test_labels = np.array(test_labels).reshape([-1, 1])
    test_acc += sess.run(accuracy, feed_dict={X: test_reviews, y: test_labels})
    n+=1
acc=test_acc/n 
print("Final accuracy:", acc)
sess.close()

Final accuracy: 0.9518413597733711


### (iii) Encode each vocabulary word as a one-hot vector. Train an MLP on the average of the onehot vectors.

In [0]:
import numpy as np
one_hot_representation = np.identity(len(english_words), dtype=np.float32)

# Same as part (a)
import string
remove_punct=str.maketrans('','',string.punctuation)

# This function converts a line of our data file into
# a tuple (x, y), where x is len(english_words)-dimensional representation (1-hot)
# of the words in a review, and y is its label.
def convert_line_to_one_hot_example(line):
    # Pull out the first character: that's our label (0 or 1)
    y = int(line[0])
    # Split the line into words using Python's split() function
    words = line[2:].translate(remove_punct).lower().split()
    # Look up the embeddings of each word, ignoring words not
    # in our pretrained vocabulary.
    embeddings = [one_hot_representation[index[w]] for w in words
                  if w in index]
    # Take the mean of the embeddings
    x = np.mean(np.vstack(embeddings), axis=0)
    return {'x': x, 'y': y, 'w':embeddings}

# Apply the function to each line in the file.
enc = 'utf-8' # This is necessary from within the singularity shell


In [0]:
# Same implementation as (a)
with open("movie-simple.txt", "r", encoding=enc) as f:
    # Now convert line to one-hot embeddings
    dataset = [convert_line_to_one_hot_example(l) for l in f.readlines()] 
import random
random.shuffle(dataset)

batch_size = 100
total_batches = len(dataset) // batch_size
train_batches = 3 * total_batches // 4
train, test = dataset[:train_batches*batch_size], dataset[train_batches*batch_size:]

In [0]:
import tensorflow as tf
tf.reset_default_graph()

# Placeholders for input
X = tf.placeholder(tf.float32, [None, len(english_words)]) # 150875-D vectors
y = tf.placeholder(tf.float32, [None, 1]) # Binary output (T/F)

# Three-layer MLP
h1 = tf.layers.dense(X, 100, tf.nn.relu)
h2 = tf.layers.dense(h1, 20, tf.nn.relu)
logits = tf.layers.dense(h2, 1)
probabilities = tf.sigmoid(logits)

# Loss and metrics
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(tf.sigmoid(logits)), y), tf.float32))

# Training using Adam !
train_step = tf.train.AdamOptimizer().minimize(loss)

# Initialization of variables
initialize_all = tf.global_variables_initializer()

In [0]:
# Train
sess = tf.InteractiveSession()
sess.run(initialize_all)
for epoch in range(250):
    for batch in range(train_batches):
        data = train[batch*batch_size:(batch+1)*batch_size]
        reviews = [sample['x'] for sample in data]
        labels  = [sample['y'] for sample in data]
        labels = np.array(labels).reshape([-1, 1])
        _, l, acc = sess.run([train_step, loss, accuracy], feed_dict={X: reviews, y: labels})
    if epoch % 10 == 0:
        print("Epoch", epoch, "Loss", l, "Acc", acc)
    random.shuffle(train)

# Evaluate on test set
test_reviews = [sample['x'] for sample in test]
test_labels  = [sample['y'] for sample in test]
test_labels = np.array(test_labels).reshape([-1, 1])
acc = sess.run(accuracy, feed_dict={X: test_reviews, y: test_labels})
print("Final accuracy:", acc)
sess.close()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch 0 Loss 0.69226265 Acc 0.5
Epoch 10 Loss 0.6821994 Acc 0.53
Epoch 20 Loss 0.68925375 Acc 0.49
Epoch 30 Loss 0.6647892 Acc 0.59
Epoch 40 Loss 0.6734055 Acc 0.52
Epoch 50 Loss 0.63869524 Acc 0.67
Epoch 60 Loss 0.6288469 Acc 0.63
Epoch 70 Loss 0.57186645 Acc 0.78
Epoch 80 Loss 0.50734836 Acc 0.84
Epoch 90 Loss 0.39258626 Acc 0.93
Epoch 100 Loss 0.36231154 Acc 0.86
Epoch 110 Loss 0.31141827 Acc 0.92
Epoch 120 Loss 0.24365845 Acc 0.96
Epoch 130 Loss 0.2425871 Acc 0.94
Epoch 140 Loss 0.24263132 Acc 0.92
Epoch 150 Loss 0.2242392 Acc 0.92
Epoch 160 Loss 0.18156032 Acc 0.95
Epoch 170 Loss 0.18917479 Acc 0.91
Epoch 180 Loss 0.112309545 Acc 0.97
Epoch 190 Loss 0.14119118 Acc 0.98
Epoch 200 Loss 0.15195604 Acc 0.94
Epoch 210 Loss 0.08990037 Acc 0.98
Epoch 220 Loss 0.18237732 Acc 0.95
Epoch 230 Loss 0.11635973 Acc 0.97
Epoch 240 Loss 0.15585178 Acc 0.94
Final accuracy: 0.9659367


### (iv) As in (iii), but use an RNN on the one-hot encodings.

In [0]:
with open("movie-simple.txt", "r",encoding=enc) as f:
    dataset = [convert_line_to_one_hot_example(l) for l in f.readlines()]
import random
random.shuffle(dataset)
batch_size = 1
total_batches = len(dataset) // batch_size
train_batches = 3 * total_batches // 4
train, test = dataset[:train_batches*batch_size], dataset[train_batches*batch_size:]

In [0]:
tf.reset_default_graph()
# sizes
n_steps = None
n_inputs = len(english_words) # 150875-D vectors
n_neurons = 50
n_outputs = 1 # binary output

# Build RNN
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_outputs])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.tanh)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
y_ = tf.layers.dense(outputs[:,-1,:], n_outputs) #logits

# Loss and metrics
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=y_)
loss = tf.reduce_mean(cross_entropy)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(tf.sigmoid(y_)), y), tf.float32))

# Training
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

In [0]:
num_epochs = 10 # CHANGED SINCE TRAINING IS TOO SLOW!
initialize_all = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(initialize_all)
l_ma=.74
acc_ma=.5

for epoch in range(num_epochs):
    for batch in range(train_batches):
        data = train[batch*batch_size:(batch+1)*batch_size]
        reviews = np.array([sample['w'] for sample in data]).reshape([1,-1,len(english_words)])
        labels  = np.array([sample['y'] for sample in data]).reshape([1,1])
        labels = np.array(labels).reshape([-1, 1])
        _, l, acc = sess.run([train_step, loss, accuracy], feed_dict={X: reviews, y: labels})
        l_ma=.99*l_ma+(.01)*l
        acc_ma=.99*acc_ma+(.01)*acc
    if epoch % 1 == 0:
        print("Epoch", epoch, "Loss", l_ma, "Acc", acc_ma)
    random.shuffle(train)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch 0 Loss 0.40502152455505647 Acc 0.796964593683877
Epoch 1 Loss 0.1662961574440515 Acc 0.9355787352018449
Epoch 2 Loss 0.12527569385690035 Acc 0.9513816659501054
Epoch 3 Loss 0.134874889028147 Acc 0.9387998497851943
Epoch 4 Loss 0.0890855157124037 Acc 0.9595905268229524
Epoch 5 Loss 0.07715123540561743 Acc 0.9806638545230753
Epoch 6 Loss 0.05880502201158864 Acc 0.9819542680566435
Epoch 7 Loss 0.1555484656472933 Acc 0.948687954174636
Epoch 8 Loss 0.1668978511977483 Acc 0.948808927511394
Epoch 9 Loss 0.03226694494256042 Acc 0.9796306662474786


In [0]:
# Evaluate on test set
test_acc=0
n=0
for sample in test:
    test_reviews = np.array([sample['w'] ]).reshape([1,-1,len(english_words)])
    test_labels  = np.array([sample['y']]).reshape([1,1])
    test_labels = np.array(test_labels).reshape([-1, 1])
    test_acc += sess.run(accuracy, feed_dict={X: test_reviews, y: test_labels})
    n+=1
acc=test_acc/n 
print("Final accuracy:", acc)
sess.close()

Final accuracy: 0.9405099150141643


### (v) Why did the word embeddings work better (hint: the word embeddings will work better…)

**Answer:**

One-hot vectors are typically high-dimensional, and in this problem, the one-hot matrix has a very large size (~150,000) and also very sparse. In contrast, word embeddings are low-dimensional (300 in this problem). Word embeddings have the ability to generalize, due to semantically similar words having similar vectors, which is not the case in one-hot vectors. In our problem, the length of the given dataset is about 1,400. Thus, the input dimension of 150,000 introduced by ont-hot embeddings is too large to generalize. Also, it costs much more computational resources.

**Time-series prediction is an important task, which is commonly approached with RNNs.**

### (vi) How does cross-validation change when considering a time-series instead of multiple instances (as in our movie reviews)? Only a description is needed.



**Answer:**

When dealing with time series, unlike the traditional k-fold cross validation, our modified cross-validation strategy/solution is to break the time series into k continuous chunks, then train on the first k sets of data and evaluate on the k+1 th set of data. However, this will not work well when we are dealing with multiple instances because one assumption is that different instances are independently distributed.  We will lose some information if we split these instances since each instance is distinct.

**Much of the strength of deep learning comes from the ability to stitch together different deep
learning modules and approaches into a larger framework. In class, we discussed training an
RNN against a loss function. Instead, CNNs, RNNs, GANs can all be combined.**

### (vii) In our previous homework assignment we considered the conditional GAN. In that case, the conditional label was known and given. Instead, consider generating images to match text. One approach could be to use an RNN to encode text to a vector that is fed to a conditional GAN (e.g. http://proceedings.mlr.press/v48/reed16.pdf). Draw a graph (but do not implement) how such a system could work. Any implementation here is completely optional, we are only looking for a description of how this could work.

Answer: 

See PDF file.

## Problem 4: SARSA Q-Learning (30 points)

**SARSA is an alternative way of learning a Q-Value function with a lot of similarity to the simpler
policy introduced during the code exercises.**

**Q-learning update rule:**

**$Q^{new}(s_t, a_t) \leftarrow (1 - \alpha)Q^{old}(s_t, a_t) + \alpha[r_t + \gamma max_aQ^{old}(s_{t+1},a)]$**

**SARSA update rule:**

**$Q^{new}(s_t, a_t) \leftarrow (1 - \alpha)Q^{old}(s_t, a_t) + \alpha[r_t + \gamma Q^{old}(s_{t+1},a_{t+1})]$**

**Unlike Q-learning, which is considered an *off-policy* network, SARSA is an *on-policy* algorithm.
When Q-learning calculates the estimated future reward, it must "guess" the future, starting
with the next action the agent will take. In Q-learning, we assume the agent will take the best
possible action. SARSA, on the other hand, uses the action that was actually taken next in the
episode we are learning from. In other words, SARSA learns from the next action he actually
took (on policy), as opposed to what the max possible Q value for the next state was (off policy).**

**Here, we will use the same setup that we discussed in class with the cart pole problem.**

In [1]:
!pip install gym

### (a) Implement a deep Q-learning approach to the cart pole problem (was done in class) using an $\epsilon$-Greedy approach

In [0]:
# Based on: https://gym.openai.com/evaluations/eval_EIcM1ZBnQW2LBaFN6FY65g/

import random
import gym
import math
import numpy as np
import tensorflow as tf
from collections import deque

In [7]:
class DQNCartPoleSolver():
    def __init__(self, n_episodes=1000, n_win_ticks=195, max_env_steps=None, gamma=1.0, epsilon=1.0, epsilon_min=0.01, epsilon_log_decay=0.995, alpha=0.01, alpha_decay=0.01, batch_size=64, monitor=False, quiet=False):
        self.memory = deque(maxlen=100000)
        self.env = gym.make('CartPole-v0')
        if monitor: self.env = gym.wrappers.Monitor(self.env, '../data/cartpole-1', force=True)
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_log_decay
        self.alpha = alpha
        self.alpha_decay = alpha_decay
        self.n_episodes = n_episodes
        self.n_win_ticks = n_win_ticks
        self.batch_size = batch_size
        self.quiet = quiet
        if max_env_steps is not None: self.env._max_episode_steps = max_env_steps

        # Init model
        self.state_ = tf.placeholder(tf.float32, shape=[None, 4])
        h = tf.layers.dense(self.state_, units=24, activation=tf.nn.tanh)
        h = tf.layers.dense(h, units=48, activation=tf.nn.tanh)
        self.Q = tf.layers.dense(h, units=2)
        
        self.Q_ = tf.placeholder(tf.float32, shape=[None, 2])
        loss = tf.losses.mean_squared_error(self.Q_, self.Q)
        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        lr = tf.train.exponential_decay(0.01, self.global_step, 0.995, 1)
        self.train_step = tf.train.AdamOptimizer(lr).minimize(loss, global_step=self.global_step)
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def choose_action(self, state, epsilon):
        return self.env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(self.sess.run(self.Q, feed_dict={self.state_: state}))

    def get_epsilon(self, t):
        return max(self.epsilon_min, min(self.epsilon, 1.0 - math.log10((t + 1) * self.epsilon_decay)))

    def preprocess_state(self, state):
        return np.reshape(state, [1, 4])

    def replay(self, batch_size):
        x_batch, y_batch = [], []
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
        for state, action, reward, next_state, done in minibatch:
            y_target = self.sess.run(self.Q, feed_dict={self.state_: state})
            y_target[0][action] = reward if done else reward + self.gamma * np.max(self.sess.run(self.Q, feed_dict={self.state_: next_state})[0])
            x_batch.append(state[0])
            y_batch.append(y_target[0])
        
        self.sess.run(self.train_step, feed_dict={self.state_: np.array(x_batch), self.Q_: np.array(y_batch)})

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def run(self):
        scores = deque(maxlen=100)

        for e in range(self.n_episodes):
            state = self.preprocess_state(self.env.reset())
            done = False
            i = 0
            while not done:
                #if e % 100 == 0 and not self.quiet:
                #    self.env.render()
                action = self.choose_action(state, self.get_epsilon(e))
                next_state, reward, done, _ = self.env.step(action)
                next_state = self.preprocess_state(next_state)
                self.remember(state, action, reward, next_state, done)
                state = next_state
                i += 1

            scores.append(i)
            mean_score = np.mean(scores)
            if mean_score >= self.n_win_ticks and e >= 100:
                if not self.quiet: print('Ran {} episodes. Solved after {} trials ✔'.format(e, e - 100))
                return e - 100
            if e % 100 == 0 and not self.quiet:
                print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))

            self.replay(self.batch_size)
        
        if not self.quiet: print('Did not solve after {} episodes 😞'.format(e))
        return e

if __name__ == '__main__':
    agent = DQNCartPoleSolver()
    agent.run()

/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


[Episode 0] - Mean survival time over last 100 episodes was 12.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 20.75 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 28.49 ticks.
[Episode 300] - Mean survival time over last 100 episodes was 92.88 ticks.
[Episode 400] - Mean survival time over last 100 episodes was 131.63 ticks.
[Episode 500] - Mean survival time over last 100 episodes was 123.88 ticks.
[Episode 600] - Mean survival time over last 100 episodes was 133.79 ticks.
[Episode 700] - Mean survival time over last 100 episodes was 141.96 ticks.
[Episode 800] - Mean survival time over last 100 episodes was 141.36 ticks.
[Episode 900] - Mean survival time over last 100 episodes was 162.49 ticks.
Did not solve after 999 episodes 😞


In [8]:
class DQNCartPoleSolver(): # REDO 
    def __init__(self, n_episodes=1000, n_win_ticks=195, max_env_steps=None, gamma=1.0, epsilon=1.0, epsilon_min=0.01, epsilon_log_decay=0.995, alpha=0.01, alpha_decay=0.01, batch_size=64, monitor=False, quiet=False):
        self.memory = deque(maxlen=100000)
        self.env = gym.make('CartPole-v0')
        if monitor: self.env = gym.wrappers.Monitor(self.env, '../data/cartpole-1', force=True)
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_log_decay
        self.alpha = alpha
        self.alpha_decay = alpha_decay
        self.n_episodes = n_episodes
        self.n_win_ticks = n_win_ticks
        self.batch_size = batch_size
        self.quiet = quiet
        if max_env_steps is not None: self.env._max_episode_steps = max_env_steps

        # Init model
        self.state_ = tf.placeholder(tf.float32, shape=[None, 4])
        h = tf.layers.dense(self.state_, units=24, activation=tf.nn.tanh)
        h = tf.layers.dense(h, units=48, activation=tf.nn.tanh)
        self.Q = tf.layers.dense(h, units=2)
        
        self.Q_ = tf.placeholder(tf.float32, shape=[None, 2])
        loss = tf.losses.mean_squared_error(self.Q_, self.Q)
        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        lr = tf.train.exponential_decay(0.01, self.global_step, 0.995, 1)
        self.train_step = tf.train.AdamOptimizer(lr).minimize(loss, global_step=self.global_step)
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def choose_action(self, state, epsilon):
        return self.env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(self.sess.run(self.Q, feed_dict={self.state_: state}))

    def get_epsilon(self, t):
        return max(self.epsilon_min, min(self.epsilon, 1.0 - math.log10((t + 1) * self.epsilon_decay)))

    def preprocess_state(self, state):
        return np.reshape(state, [1, 4])

    def replay(self, batch_size):
        x_batch, y_batch = [], []
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
        for state, action, reward, next_state, done in minibatch:
            y_target = self.sess.run(self.Q, feed_dict={self.state_: state})
            y_target[0][action] = reward if done else reward + self.gamma * np.max(self.sess.run(self.Q, feed_dict={self.state_: next_state})[0])
            x_batch.append(state[0])
            y_batch.append(y_target[0])
        
        self.sess.run(self.train_step, feed_dict={self.state_: np.array(x_batch), self.Q_: np.array(y_batch)})

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def run(self):
        scores = deque(maxlen=100)

        for e in range(self.n_episodes):
            state = self.preprocess_state(self.env.reset())
            done = False
            i = 0
            while not done:
                #if e % 100 == 0 and not self.quiet:
                #    self.env.render()
                action = self.choose_action(state, self.get_epsilon(e))
                next_state, reward, done, _ = self.env.step(action)
                next_state = self.preprocess_state(next_state)
                self.remember(state, action, reward, next_state, done)
                state = next_state
                i += 1

            scores.append(i)
            mean_score = np.mean(scores)
            if mean_score >= self.n_win_ticks and e >= 100:
                if not self.quiet: print('Ran {} episodes. Solved after {} trials ✔'.format(e, e - 100))
                return e - 100
            if e % 100 == 0 and not self.quiet:
                print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))

            self.replay(self.batch_size)
        
        if not self.quiet: print('Did not solve after {} episodes 😞'.format(e))
        return e

if __name__ == '__main__':
    agent = DQNCartPoleSolver(n_episodes=1500)
    agent.run()

/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


[Episode 0] - Mean survival time over last 100 episodes was 23.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 22.32 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 58.41 ticks.
[Episode 300] - Mean survival time over last 100 episodes was 91.81 ticks.
[Episode 400] - Mean survival time over last 100 episodes was 124.66 ticks.
[Episode 500] - Mean survival time over last 100 episodes was 140.79 ticks.
[Episode 600] - Mean survival time over last 100 episodes was 112.16 ticks.
[Episode 700] - Mean survival time over last 100 episodes was 148.73 ticks.
[Episode 800] - Mean survival time over last 100 episodes was 156.81 ticks.
[Episode 900] - Mean survival time over last 100 episodes was 135.08 ticks.
[Episode 1000] - Mean survival time over last 100 episodes was 155.75 ticks.
[Episode 1100] - Mean survival time over last 100 episodes was 94.18 ticks.
[Episode 1200] - Mean survival time over last 100 episodes was 135.42 ticks.
[Episode 1300] - 

### (b) Implement a SARSA approach with a deep network to solve the cart pole problem using an $\epsilon$-Greedy approach

In [10]:
class SARSACartPoleSolver():
    def __init__(self, n_episodes=1000, n_win_ticks=195, max_env_steps=None, gamma=1.0, epsilon=1.0, epsilon_min=0.01, epsilon_log_decay=0.995, alpha=0.01, alpha_decay=0.01, batch_size=64, monitor=False, quiet=False):
        self.memory = deque(maxlen=100000)
        self.env = gym.make('CartPole-v0')
        if monitor: self.env = gym.wrappers.Monitor(self.env, '../data/cartpole-1', force=True)
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_log_decay
        self.alpha = alpha
        self.alpha_decay = alpha_decay
        self.n_episodes = n_episodes
        self.n_win_ticks = n_win_ticks
        self.batch_size = batch_size
        self.quiet = quiet
        if max_env_steps is not None: self.env._max_episode_steps = max_env_steps

        # Init model
        self.state_ = tf.placeholder(tf.float32, shape=[None, 4])
        h = tf.layers.dense(self.state_, units=24, activation=tf.nn.tanh)
        h = tf.layers.dense(h, units=48, activation=tf.nn.tanh)
        self.Q = tf.layers.dense(h, units=2)
        
        self.Q_ = tf.placeholder(tf.float32, shape=[None, 2])
        loss = tf.losses.mean_squared_error(self.Q_, self.Q)
        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        lr = tf.train.exponential_decay(0.01, self.global_step, 0.995, 1)
        self.train_step = tf.train.AdamOptimizer(lr).minimize(loss, global_step=self.global_step)
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
    # Modification: Add a_t+1 term (next action)
    def remember(self, state, action, reward, next_state, next_action, done):
        self.memory.append((state, action, reward, next_state, next_action, done))

    def choose_action(self, state, epsilon):
        return self.env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(self.sess.run(self.Q, feed_dict={self.state_: state}))

    def get_epsilon(self, t):
        return max(self.epsilon_min, min(self.epsilon, 1.0 - math.log10((t + 1) * self.epsilon_decay)))

    def preprocess_state(self, state):
        return np.reshape(state, [1, 4])

    def replay(self, batch_size):
        x_batch, y_batch = [], []
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
        for state, action, reward, next_state, next_action, done in minibatch: # Modification: Add a_t+1 term (next action)
            y_target = self.sess.run(self.Q, feed_dict={self.state_: state})
            # Modification: Q^old(s_t+1, a_t+1)
            y_target[0][action] = reward if done else reward + self.gamma * self.sess.run(self.Q, feed_dict={self.state_: next_state})[0][next_action]
            x_batch.append(state[0])
            y_batch.append(y_target[0])
        
        self.sess.run(self.train_step, feed_dict={self.state_: np.array(x_batch), self.Q_: np.array(y_batch)})

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def run(self):
        scores = deque(maxlen=100)

        for e in range(self.n_episodes):
            state = self.preprocess_state(self.env.reset())
            # Modification: Add initial conditions a0
            action = self.choose_action(state, self.get_epsilon(e))
            
            done = False
            i = 0
            while not done:
                #if e % 100 == 0 and not self.quiet:
                #    self.env.render()
                # action = self.choose_action(state, self.get_epsilon(e))
                next_state, reward, done, _ = self.env.step(action)
                next_state = self.preprocess_state(next_state)
                # Modification: Choose next action based on next state
                action_next = self.choose_action(next_state, self.get_epsilon(e))
                self.remember(state, action, reward, next_state, action_next, done)
                state = next_state
                action = action_next # Update action as well
                i += 1

            scores.append(i)
            mean_score = np.mean(scores)
            if mean_score >= self.n_win_ticks and e >= 100:
                if not self.quiet: print('Ran {} episodes. Solved after {} trials ✔'.format(e, e - 100))
                return e - 100
            if e % 100 == 0 and not self.quiet:
                print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))

            self.replay(self.batch_size)
        
        if not self.quiet: print('Did not solve after {} episodes 😞'.format(e))
        return e

if __name__ == '__main__':
    agent = SARSACartPoleSolver()
    agent.run()

/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


[Episode 0] - Mean survival time over last 100 episodes was 19.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 11.19 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 23.51 ticks.
[Episode 300] - Mean survival time over last 100 episodes was 98.03 ticks.
[Episode 400] - Mean survival time over last 100 episodes was 181.52 ticks.
Ran 496 episodes. Solved after 396 trials ✔


### (c) Evaluate the impact of $\gamma$ in the cart pole problem. How important is this parameter? How does it affect stability and learning?

In [12]:
for gamma in [0, 0.25, 0.5, 0.75, 1]:
    print("*************Gamma selected = {}*************\n".format(gamma))
    print("*************Solving using DQN***************")
    agent1 = DQNCartPoleSolver(n_episodes=1500, gamma=gamma)
    agent1.run()
    print("*************Solving using SARSA*************")
    agent2 = SARSACartPoleSolver(n_episodes=1500, gamma=gamma)
    agent2.run()

*************Gamma selected = 0*************

*************Solving using DQN***************


/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


[Episode 0] - Mean survival time over last 100 episodes was 21.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 15.46 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 9.27 ticks.
[Episode 300] - Mean survival time over last 100 episodes was 9.44 ticks.
[Episode 400] - Mean survival time over last 100 episodes was 9.55 ticks.
[Episode 500] - Mean survival time over last 100 episodes was 9.5 ticks.
[Episode 600] - Mean survival time over last 100 episodes was 9.55 ticks.
[Episode 700] - Mean survival time over last 100 episodes was 9.58 ticks.
[Episode 800] - Mean survival time over last 100 episodes was 9.43 ticks.
[Episode 900] - Mean survival time over last 100 episodes was 9.49 ticks.
[Episode 1000] - Mean survival time over last 100 episodes was 9.97 ticks.
[Episode 1100] - Mean survival time over last 100 episodes was 10.12 ticks.
[Episode 1200] - Mean survival time over last 100 episodes was 9.45 ticks.
[Episode 1300] - Mean survival time 

**Answer:**

$\gamma$ is a very important parameter for both of our reinforcement learning networks. Here, I selected 5 different values for $\gamma$ ranges from 0 to 1 to test on both a deep Q-learning network and a SARSA network. Based on the results, we can see that both networks could solve the problem only when $\gamma$ is exactly equal to 1. We know that $\gamma = 1$ means that the agent cares about long-term rewards. A relatively small $\gamma$ value (< 0.5) produces stable poor performance since the agent learns very slowly and could not find a solution in the given time. If we increase the value, the stability of output becomes worse, as we can see from the outputs have a wider range. Also, in general, SARSA is more stable than DQN and could run faster.

### (d) Evaluate the impact of $\epsilon$ on the learning over the feasible range (0-1). What values seem reasonable?

In [13]:
for e in [0, 0.25, 0.5, 0.75, 1]:
    print("*************Epsilon selected = {}*************\n".format(e))
    print("*************Solving using DQN***************")
    agent1 = DQNCartPoleSolver(n_episodes=1500, epsilon=e, epsilon_min=0.0)
    agent1.run()
    print("*************Solving using SARSA*************")
    agent2 = SARSACartPoleSolver(n_episodes=1500, epsilon=e, epsilon_min=0.0)
    agent2.run()

*************Epsilon selected = 0*************

*************Solving using DQN***************


/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


[Episode 0] - Mean survival time over last 100 episodes was 39.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 64.27 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 30.2 ticks.
[Episode 300] - Mean survival time over last 100 episodes was 131.58 ticks.
[Episode 400] - Mean survival time over last 100 episodes was 116.85 ticks.
[Episode 500] - Mean survival time over last 100 episodes was 129.87 ticks.
[Episode 600] - Mean survival time over last 100 episodes was 155.44 ticks.
[Episode 700] - Mean survival time over last 100 episodes was 157.77 ticks.
[Episode 800] - Mean survival time over last 100 episodes was 106.87 ticks.
Ran 897 episodes. Solved after 797 trials ✔
*************Solving using SARSA*************
[Episode 0] - Mean survival time over last 100 episodes was 9.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 9.39 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 9.41 ticks.
[Episode 300] 

**Answer:**

$\epsilon$ controls the model to either explore or exploit. Based on the test results, unlike $\gamma$, the agents could solve the problem with different initial values of $\epsilon$ ($\epsilon$ has a exponential decay term), so it is not as critical as $\gamma$ for our RL models to find a solution in given time. More specifically, I assume that an initial $\epsilon$ value that is larger than 0.5 seems reasonable since both agents perform well in this case, and also run faster.

### (e) Pick one other small agent from the OpenAI gym (https://gym.openai.com/envs/#classic_control) and apply the same techniques 

In [47]:
env = gym.make("MountainCar-v0")


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [25]:
env.observation_space

Box(2,)

In [26]:
env.action_space

Discrete(3)

In [3]:
# Code based on https://gitlab.oit.duke.edu/dec18/intro_to_deep_learning/blob/master/lectures/22_Reinforcement_Learning.ipynb
# Initialization rules based on https://github.com/openai/gym/wiki/MountainCar-v0
# Some network architecture based on https://medium.com/@ts1829/solving-mountain-car-with-q-learning-b77bf71b1de2
# USING SARSA METHOD

class SARSAMountainCarSolver():
    def __init__(self, n_episodes=1000, n_win_ticks=195, max_env_steps=None, gamma=0.99, epsilon=0.9, epsilon_min=0.01, epsilon_log_decay=0.995, alpha=0.01, alpha_decay=0.01, batch_size=64, monitor=False, quiet=False):
        self.memory = deque(maxlen=10000)
        self.env = gym.make('MountainCar-v0')
        if monitor: self.env = gym.wrappers.Monitor(self.env, '../data/mountaincar-1', force=True)
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_log_decay
        self.alpha = alpha
        self.alpha_decay = alpha_decay
        self.n_episodes = n_episodes
        self.n_win_ticks = n_win_ticks
        self.batch_size = batch_size
        self.quiet = quiet
        if max_env_steps is not None: self.env._max_episode_steps = max_env_steps
        self.init_lr = 0.001
        # Init model
        self.state_ = tf.placeholder(tf.float32, shape=[None, 2])
        h = tf.layers.dense(self.state_, units=24, activation=tf.nn.tanh, kernel_initializer=tf.truncated_normal_initializer)
        h = tf.layers.dense(h, units=48, activation=tf.nn.tanh)
        self.Q = tf.layers.dense(h, units=3)
        
        self.Q_ = tf.placeholder(tf.float32, shape=[None, 3])
        loss = tf.losses.mean_squared_error(self.Q_, self.Q)
        
        self.lr = tf.placeholder(tf.float32, shape=[])
        self.train_step = tf.train.GradientDescentOptimizer(learning_rate=self.lr).minimize(loss)
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
    # Modification: Add a_t+1 term (next action)
    def remember(self, state, action, reward, next_state, next_action, done):
        self.memory.append((state, action, reward, next_state, next_action, done))

    def choose_action(self, state, epsilon):
        return self.env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(self.sess.run(self.Q, feed_dict={self.state_: state, self.lr: self.init_lr}))

    def get_epsilon(self, t):
        return self.epsilon

    def preprocess_state(self, state):
        return np.reshape(state, [1, 2])

    def replay(self, batch_size):
        x_batch, y_batch = [], []
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
        for state, action, reward, next_state, next_action, done in minibatch: # Modification: Add a_t+1 term (next action)
            y_target = self.sess.run(self.Q, feed_dict={self.state_: state, self.lr:self.init_lr})
            # Modification: Q^old(s_t+1, a_t+1)
            y_target[0][action] = reward if done else reward + self.gamma * self.sess.run(self.Q, feed_dict={self.state_: next_state, self.lr:self.init_lr})[0][next_action]
            x_batch.append(state[0])
            y_batch.append(y_target[0])
        
        self.sess.run(self.train_step, feed_dict={self.state_: np.array(x_batch), self.Q_: np.array(y_batch), self.lr:self.init_lr})

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def run(self):
        scores = deque(maxlen=100)
        steps = 300
        for e in range(self.n_episodes):
            state = self.preprocess_state(self.env.reset())
            # Modification: Add initial conditions a0
            action = self.choose_action(state, self.get_epsilon(e))
            max_position = -100
            done = False
            i = 0
            #while not done:
            for s in range(steps):
                next_state, reward, done, _ = self.env.step(action) # Step forward and receive next state and reward
                reward = next_state[0]-0.5  # Adjust reward based on car position
                if next_state[0] > max_position: # Keep track of max position
                    max_position = next_state[0] 
                if next_state[0] >= 0.5: # Adjust reward for task completion
                    reward += 1
                if done:
                    if next_state[0] >= 0.5: # On successful epsisodes, adjust the following parameters
                        self.epsilon *= self.epsilon_log_decay # Adjust epsilon
                        self.init_lr *= 0.9 # Adjust learning rate
                        
                next_state = self.preprocess_state(next_state)
                # Modification: Choose next action based on next state
                next_action = self.choose_action(next_state, self.get_epsilon(e))
                self.remember(state, action, reward, next_state, next_action, done)
                
                state = next_state
                action = next_action # Update action as well
                i += 1
            # Record history
            scores.append(max_position)
            max_score = np.max(scores)
            if max_score >= 0.5 and e >= 100:
                if not self.quiet: print('Ran {} episodes. Solved after {} trials ✔'.format(e, e - 100))
                return e - 100
            if e % 10 == 0 and not self.quiet:
                print('[Episode {}] - best score {}'.format(e,max_score))

            self.replay(self.batch_size)
        
        if not self.quiet: print('Did not solve after {} episodes 😞'.format(e))
        return e

if __name__ == '__main__':
    agent = SARSAMountainCarSolver(n_episodes=1500)
    agent.run()

/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[Episode 0] - best score -0.24105732457323645
[Episode 10] - best score -0.16222821278553412
[Episode 20] - best score -0.16222821278553412
[Episode 30] - best score -0.16222821278553412
[Episode 40] - best score -0.16222821278553412
[Episode 50] - best score -0.1274514999041625
[Episode 60] - best score -0.1274514999041625
[Episode 70] - best score -0.1274514999041625
[Episode 80] - best score -0.1274514999041625
[Episode 90] - best score -0.1274514999041625
[Episode 100] - best score -0.09157336651680505
[Episode 110] - best score 0.09608384058276344
[Episode 120] - best score 0.10661531639204574
[Episode 130] - best score 0.10661531639204574
[Episode 140] - best score 0.10661531639204574
[Episode 150] - best score 0.22521521469648118
[Episode 160] - best score 0.32939201220800196
[Episode 170] - best score 0.32939201220800196
[Episode 180] - best score 0.34080127717802156
[Episode 190]

## Problem 5: Bookkeeping (5 points)

### (a) How many hours did this assignment take you? (There is NO correct answer here, this is just an information gathering exercise)

About 30 hours in total. This HW is long.

### (b) Verify that you adhered to the Duke Community Standard in this assignment
(https://studentaffairs.duke.edu/conduct/about-us/duke-community-standard).

I adhered to the Duke Community Standard in the completion of this
assignment

Yifan Li